- Overview of example system with data interfaces
- air source heat pump
- What inputs/information do we need? e.g. COP, demand time series, price electricity, ...
- How can we generate those inputs

1. solph base setup

- energy system
- buses
- electricity source

In [ ]:
import oemof.solph as solph

bus_electricity = solph.Bus(label="electricity")
bus_heat_35C = solph.Bus(label="heat 35C")

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={bus_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

heating_rod = solph.components.Transformer(
    label="heating rod",
    inputs={bus_electricity: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow()},
)

thermal_storage = solph.components.GenericStorage(
    label='thermal_storage',
    inputs={bus_heat_35C: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow()},
    loss_rate=0.02,
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

In [ ]:
from workshop.model.utilities import load_input_data


input_data = load_input_data().head(24*3)

demand = input_data["Heat load (kW)"]

print(demand)

heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={bus_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)


2. Heat pump

- we need the COP, where to get that? Datasheet viessmann
- COP depends on temperature, we only know a single temperature value
- Use Carnot method with constant efficiency assumption

- Code for COP vs. temperature calculation


In [ ]:
heat_pump = solph.components.Transformer(
    label="heat pump",
    inputs={bus_electricity: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow(nominal_value=hp_thermal_power)},
    conversion_factors={
        bus_electricity: 1 / cop,
        bus_heat_35C: 1,
    },
)

- Aspect of reference temperature for COP
- 7;35 vs. 2;40
- show plot comparing both COPs vs. ambient temperature or both COPs in time series

3. Demand

- format of the demand required for solph
- we use trivial model, give formula here, code cell for defining demand

4. Storage

- what data do we need, how do we calculate the storage content value?

- Run the model, show:
  - duration curve, dispatch, electricity consumption

- Then TESPy model to generate COP, reconfigure the heat pump in solph and run again.

Introduce minimal load, reconfigure heat pump for that, rerun and make plots.

Get part load dependent COP from TESPy, reconfigure heat pump, rerun and make plots.